# Monowai CC tests

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from glob import glob
import os
import xcorr

In [3]:
%matplotlib notebook

## Merge files

In [4]:
dest = '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full'

In [5]:
ncfiles = []
for root, dirs, files in os.walk(dest):
    path = root.split(os.sep)
    # print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        # print(len(path) * '---', file)
        if file[1] != '.' and file[-3:] == '.nc':
            ncfiles += [os.path.join(root,file)]

In [6]:
sorted(ncfiles)

['/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H03S1..EDH-IU.RAR.10.BHR/IM.H03S1..EDH-IU.RAR.10.BHR.2015.015.nc',
 '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H03S1..EDH-IU.RAR.10.BHR/IM.H03S1..EDH-IU.RAR.10.BHR.2015.016.nc',
 '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H03S1..EDH-IU.RAR.10.BHZ/IM.H03S1..EDH-IU.RAR.10.BHZ.2015.015.nc',
 '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H03S1..EDH-IU.RAR.10.BHZ/IM.H03S1..EDH-IU.RAR.10.BHZ.2015.016.nc',
 '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H10N1..EDH-IU.RAR.10.BHR/IM.H10N1..EDH-IU.RAR.10.BHR.2015.015.nc',
 '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H10N1..EDH-IU.RAR.10.BHR/IM.H10N1..EDH-IU.RAR.10.BHR.2015.016.nc',
 '/Users/psmets/Documents/Research/_studies/_hydro/monowai_remote/tests/full/IM.H10N1..EDH-IU.RAR.10.BHZ/IM.H10N1..EDH

In [7]:
dsets = xcorr.merge(sorted(ncfiles), extract=False, fast=True)

In [9]:
xcorr.bias_correct(dsets)

In [10]:
dsets

<xarray.Dataset>
Dimensions:      (lag: 1080000, pair: 4, time: 16)
Coordinates:
  * pair         (pair) object 'IM.H03S1..EDH-IU.RAR.10.BHR' ... 'IM.H10N1..EDH-IU.RAR.10.BHZ'
  * time         (time) datetime64[ns] 2015-01-15 ... 2015-01-16T21:00:00
  * lag          (lag) timedelta64[ns] 00:00:00 ... 05:59:59.980000
Data variables:
    distance     (pair) float64 7.315e+03 7.315e+03 1.399e+04 1.399e+04
    status       (pair, time) int8 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    hash         (pair, time) object 'b1d5dcafe33602d7c4381a0e2a69a9fcab3480cba5faa0baa8ae730321b90f60' ... '149cb3e057e13a435b393c24622c59c19cb8accc0256ddf1d407e97dde8e19a9'
    pair_offset  (pair, time) timedelta64[ns] 00:00:00.005500 ... 00:00:00.005500
    time_offset  (pair, time) timedelta64[ns] 00:00:00 00:00:00 ... 00:00:00
    cc           (pair, time, lag) float32 0.0011199159 ... 0.0005072624
    w            (lag) float64 1.0 1.0 1.0 1.0 1.0 ... 1.333 1.333 1.333 1.333
Attributes:
    title:                 Monowai Volcanic Centre Crosscorrelations - 2015.0...
    institution:           Delft University of Technology, Department of Geos...
    author:                Pieter Smets - P.S.M.Smets@tudelft.nl
    source:                CTBTO/IMS hydroacoustic data and IRIS/USGS seismic...
    history:               Merged @ 2020-03-31 06:27:35.836320, Bias correcte...
    references:            Bendat, J. Samuel, & Piersol, A. Gerald. (1971). R...
    comment:               n/a
    Conventions:           CF-1.9
    xcorr_version:         0.1.dev77+g6f8fc5f.d20200326
    dependency_versions:   json-2.0.9, numpy-1.16.4, obspy-1.1.0, pandas-1.0....
    sha256_hash_metadata:  11c0ed1f5e06e09b670b066a16a50f9cd941cf3952f671830b...

In [11]:
vel = dict(min=1.45, max=1.50)

In [65]:
signal_mask = xcorr.signal.multi_mask_coord(
    primary=dsets.lag, secondary=dsets.distance,
    # lower=1/vel['max'], upper=1/vel['min'],
    lower=vel['min'], upper=vel['max'], invert=True,
)

In [66]:
signal_mask.history

'xcorr-0.1.dev84+g676dada.d20200327: multi_mask_coord({"primary":"lag","secondary":"distance","lower":1.45,"upper":1.5,"invert":true,"name":"mask_lag_distance"})'

In [57]:
noise_mask = xcorr.signal.mask_coord(
    coord=dsets.lag, lower=.2, scalar=dsets.time.window_length, invert=False,
)

In [54]:
repr(None)

'None'

In [56]:
noise_mask.history

'xcorr-0.1.dev84+g676dada.d20200327: mask_coord({"coord":"lag","lower":0.2,"upper":null,"scalar":86400.0,"invert":false,"name":"mask_lag","to_where":false,"**kwargs":{}})'

In [46]:
xcorr.signal.snr(dsets.cc,signal=signal_mask,noise=noise_mask,dim='lag')

NameError: name 'noise_mask' is not defined

In [ ]:
_.history

In [ ]:
dsets.distance[0]

In [ ]:
xcorr

In [ ]:
dsets['test']=dsets.cc.where(mask)

In [ ]:
dsets.history

In [ ]:
dsets

In [ ]:
plt.figure(figsize=[9,4])
test.loc[{'time':dsets.time[8]}].plot.line(x='lag',add_legend=False)

In [ ]:
plt.figure(figsize=[9,4])
dsets.cc.loc[{'pair':dsets.pair[0]}].plot.line(x='lag',add_legend=False)

In [ ]:
plt.figure(figsize=[9,4])
dsets.cc.loc[{'pair':dsets.pair[1]}].plot.line(x='lag',add_legend=False)